<a href="https://colab.research.google.com/github/IvanLudvig/eggcounting/blob/master/eggcounting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import json

In [ ]:
with open('/content/drive/MyDrive/eggcounting/data0.json') as f:
    data = json.load(f)['messages']
    df = pd.DataFrame(data)

In [ ]:
df = df[['id', 'timestamp', 'content', 'author']]

In [ ]:
df.head()

,id,timestamp,content,author
0,886460807087550536,2021-09-12T03:58:46.055+00:00,1,"{'id': '788951038971215924', 'name': 'thelegen..."
1,886460808425537597,2021-09-12T03:58:46.374+00:00,2,"{'id': '788951038971215924', 'name': 'thelegen..."
2,886460810455580732,2021-09-12T03:58:46.858+00:00,3,"{'id': '788951038971215924', 'name': 'thelegen..."
3,886460811722227713,2021-09-12T03:58:47.16+00:00,4,"{'id': '788951038971215924', 'name': 'thelegen..."
4,886460813790023730,2021-09-12T03:58:47.653+00:00,5,"{'id': '788951038971215924', 'name': 'thelegen..."


In [ ]:
df['user'] = df['author'].map(lambda x: x['name'])

In [ ]:
df.head()

,id,timestamp,content,author,user
0,886460807087550536,2021-09-12T03:58:46.055+00:00,1,"{'id': '788951038971215924', 'name': 'thelegen...",thelegendary08
1,886460808425537597,2021-09-12T03:58:46.374+00:00,2,"{'id': '788951038971215924', 'name': 'thelegen...",thelegendary08
2,886460810455580732,2021-09-12T03:58:46.858+00:00,3,"{'id': '788951038971215924', 'name': 'thelegen...",thelegendary08
3,886460811722227713,2021-09-12T03:58:47.16+00:00,4,"{'id': '788951038971215924', 'name': 'thelegen...",thelegendary08
4,886460813790023730,2021-09-12T03:58:47.653+00:00,5,"{'id': '788951038971215924', 'name': 'thelegen...",thelegendary08


In [ ]:
df.groupby(['user']).size()

user
0-0                        103
Brendan7625                100
Dawidluna                  312
FortniteDude70             662
GermanMgr14 [ARG]            1
Grinch the mean one         34
Keel                         1
Lv.99 Kira                  51
Pranoodles🍜 (she/her)     2505
Solarϟ                       2
Trial-Name                   6
Whit4You                 22956
anonymousdeveloper           2
ben1996123                  18
brainlet                 19928
const                     3892
core                       511
country croocks            202
nodu                       295
ob69jl                    1715
puds <3                      1
purple [she/they]          221
striker                   1571
thelegendary08            3606
upvoid                     253
dtype: int64

In [ ]:
df.head()

,id,timestamp,content,author,user
0,886460807087550536,2021-09-12T03:58:46.055+00:00,1,"{'id': '788951038971215924', 'name': 'thelegen...",thelegendary08
1,886460808425537597,2021-09-12T03:58:46.374+00:00,2,"{'id': '788951038971215924', 'name': 'thelegen...",thelegendary08
2,886460810455580732,2021-09-12T03:58:46.858+00:00,3,"{'id': '788951038971215924', 'name': 'thelegen...",thelegendary08
3,886460811722227713,2021-09-12T03:58:47.16+00:00,4,"{'id': '788951038971215924', 'name': 'thelegen...",thelegendary08
4,886460813790023730,2021-09-12T03:58:47.653+00:00,5,"{'id': '788951038971215924', 'name': 'thelegen...",thelegendary08
...,...,...,...,...,...
56022,892996985773236255,2021-09-30T04:51:12.49+00:00,55956,"{'id': '372494574184300565', 'name': 'Keel', '...",Keel
57814,893139993927749702,2021-09-30T14:19:28.291+00:00,57746,"{'id': '285518936219582474', 'name': 'Trial-Na...",Trial-Name
57815,893140013980717056,2021-09-30T14:19:33.072+00:00,57747,"{'id': '285518936219582474', 'name': 'Trial-Na...",Trial-Name
57816,893140018892251156,2021-09-30T14:19:34.243+00:00,57748,"{'id': '285518936219582474', 'name': 'Trial-Na...",Trial-Name
